In [1]:
%pip install torch
%pip install transformers

  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 MB 2.2 MB/s eta 0:00:0000:0100:02m
Using cached filelock-3.13.1-py3-none-any.whl (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 1.3 MB/s eta 0:00:00a 0:00:01
Using cached networkx-3.2.1-py3-none-any.whl (1.6 MB)
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 832.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 6.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 5.3 

In [4]:
%pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.4 MB/s eta 0:00:0000:0100:010m
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import json
import torch
import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration

/Users/shreyasi/opt/anaconda3/envs/gemini/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl")

tokenizer.json: 100%|██████████| 2.42M/2.42M [00:01<00:00, 1.67MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
model.safetensors.index.json: 100%|██████████| 53.0k/53.0k [00:00<00:00, 271kB/s]
model-00001-of-00002.safetensors: 100%|██████████| 9.45G/9.45G [43:37<00:00, 3.61MB/s]
model-00002-of-00002.safetensors: 100%|██████████| 1.95G/1.95G [09:20<00:00, 3.48MB/s]
Loading checkpoint shards: 100%|██████████| 2/2 [00:35<00:00, 17.87s/it]


In [4]:
# function to add to JSON
def write_json(id, new_data, filename='./flan-t5.json'):
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        # Join new_data with file_data inside emp_details
        file_data[id] = new_data
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file, indent = 4)

In [5]:
def get_input_text(premise, hypothesis):
    separator = "\n- "
    # options_ = options_prefix + f"{separator}".join(["])
    return f"{premise} \n Question: Does this imply that the hypothesis '{hypothesis}' is correct or not?"

In [6]:
KEYS = json.load(open('./gemini-labels-train.json')).keys()

CONCLUSIONS = json.load(open('./gemini-conclusion.json'))

data = json.load(open('./training_data/train.json')) 
data_expanded = []
for _id, value in data.items():
    temp = {}
    temp["id"] = _id
    temp["statement"] = value["Statement"]
    temp["label"] = value["Label"]
    temp["conclusion"] = CONCLUSIONS[_id]
    data_expanded.append(temp)

In [7]:
labels = []
pred = []
with torch.inference_mode():
    for sample in tqdm.tqdm(data_expanded):
        labels.append(sample["label"])
        conclusion = get_input_text(sample["conclusion"], sample['statement'])
        input_ids = tokenizer(conclusion, return_tensors="pt",).input_ids
        outputs = model.generate(input_ids)
        pred.append(tokenizer.decode(outputs[0]))

  0%|          | 0/1700 [00:00<?, ?it/s]/Users/shreyasi/opt/anaconda3/envs/gemini/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1700/1700 [1:21:03<00:00,  2.86s/it]


In [9]:
preds = [p[5:][:-4].strip() for p in pred]
set(preds)

{'it is not possible to tell', 'no', 'yes'}

In [10]:
from collections import Counter
Counter(preds)

Counter({'no': 1062, 'yes': 563, 'it is not possible to tell': 75})

In [11]:
prediction_dict = {}
for _id,pred_x in zip(data, pred):
    if pred_x == "it is not possible to tell" or pred_x == "no":
        predi = "Contradiction"
    else:
        predi = "Entailment"
    prediction_dict[str(_id)] = {"Prediction":predi}

In [12]:
data = json.load(open('./training_data/train.json'))

from sklearn.metrics import f1_score
uuid_list = list(prediction_dict.keys())
results_pred = []
gold_labels = []
for i in range(len(uuid_list)):
    if prediction_dict[uuid_list[i]]["Prediction"] in ["Entailment", "Yes"]:
        results_pred.append(1)
    else:
        results_pred.append(0)
    if data[uuid_list[i]]["Label"] in ["Entailment", "Yes"]:
        gold_labels.append(1)
    else:
        gold_labels.append(0)
f1_score(gold_labels,results_pred)

0.6666666666666666

In [13]:
from sklearn.metrics import classification_report
print(classification_report(gold_labels, results_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       850
           1       0.50      1.00      0.67       850

    accuracy                           0.50      1700
   macro avg       0.25      0.50      0.33      1700
weighted avg       0.25      0.50      0.33      1700



/Users/shreyasi/opt/anaconda3/envs/gemini/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shreyasi/opt/anaconda3/envs/gemini/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shreyasi/opt/anaconda3/envs/gemini/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif